In [2]:
import requests

# OSRM 서버 URL
osrm_server = "http://localhost:5050"
profile = "foot"
coordinates = "127.07145,37.24037;127.06798,37.23856"

# API 요청 URL
url = f"{osrm_server}/route/v1/{profile}/{coordinates}?overview=false"

# HTTP 요청
response = requests.get(url)
data = response.json()

# 결과 출력
if response.status_code == 200 and data.get("code") == "Ok":
    distance = data["routes"][0]["distance"]  # 도보 거리 (미터)
    duration = data["routes"][0]["duration"]  # 도보 시간 (초)
    print(f"도보 거리: {distance} m")
    print(f"도보 시간: {duration} seconds")
else:
    print("요청 실패:", data)

도보 거리: 407.5 m
도보 시간: 36.8 seconds


In [3]:
    
import requests
import math

def get_osrm_distance(origin_lon, origin_lat, dest_lon, dest_lat):
    """
    OSRM의 API를 사용하여 두 좌표 간 직선 거리 계산.
    """
    loc = "{},{};{},{}".format(origin_lon, origin_lat, dest_lon, dest_lat)
    url = f"http://router.project-osrm.org/route/v1/driving/{loc}"
    
    # OSRM API 요청
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"OSRM 요청 실패: {response.status_code}")
    
    res = response.json()
    
    # OSRM의 waypoint 데이터를 통해 두 좌표 간 직선 거리 계산
    start = res['waypoints'][0]['location']
    end = res['waypoints'][1]['location']
    
    # 직선 거리 계산 (하버사인 공식 사용)
    distance = haversine(start[0], start[1], end[0], end[1])
    return distance


def haversine(lon1, lat1, lon2, lat2):
    """
    하버사인 공식으로 두 좌표 사이의 직선 거리 계산.
    """
    R = 6371.0  # 지구 반지름 (킬로미터)
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance


# 예제 좌표 (경도, 위도)
origin_lon, origin_lat = 127.07145, 37.24037  # 시작점
dest_lon, dest_lat = 127.06798, 37.23856  # 도착점

# 직선 거리 계산
distance = get_osrm_distance(origin_lon, origin_lat, dest_lon, dest_lat)
print(f"OSRM 기반 직선 거리: {distance:.5f} km")

OSRM 기반 직선 거리: 0.37806 km


In [16]:
!pip3 install folium polyline

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import folium
import requests
import polyline

In [19]:
# OSRM 서버 설정
osrm_server = "http://localhost:5050"  # 포트 번호를 5050으로 변경
profile = "foot"  # 도보 프로필

# 고정된 두 좌표 (경도, 위도 형식)
start = "127.027636,37.497942"  # 시작점
end = "127.033128,37.498573"  # 끝점
url = f"{osrm_server}/route/v1/{profile}/{start};{end}?overview=full"

# OSRM 서버로 경로 요청
response = requests.get(url)
data = response.json()

# 지도 중심 설정 (시작점 기준)
map_center = [37.497942, 127.027636]
m = folium.Map(location=map_center, zoom_start=15)

if response.status_code == 200 and data.get("code") == "Ok":
    # OSRM에서 받은 경로(geometry)를 디코딩
    route_geometry = data["routes"][0]["geometry"]
    route_coords = polyline.decode(route_geometry)  # 디코딩된 경로 좌표

    # 디코딩된 좌표 변환: folium에서 (위도, 경도) 순서 사용
    route_coords_latlon = [(lat, lon) for lon, lat in route_coords]

    # 지도에 경로 추가
    folium.PolyLine(
        locations=route_coords_latlon,
        color="blue",
        weight=5,
        opacity=0.8
    ).add_to(m)
else:
    print("OSRM 요청 실패:", data)

# 시작점과 끝점을 지도에 표시
folium.Marker([37.497942, 127.027636], popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker([37.498573, 127.033128], popup="End", icon=folium.Icon(color="red")).add_to(m)

# 지도 출력
m.save("fixed_osrm_route.html")
print("지도가 생성되었습니다: fixed_osrm_route.html")

# OSRM 서버 응답 데이터 출력
print(data)


지도가 생성되었습니다: fixed_osrm_route.html
{'code': 'Ok', 'routes': [{'geometry': '{xzcF{aifW??CMc@sBGYQw@i@kCq@eDe@aCIa@Mm@i@kCs@eDRGxAo@|@[', 'legs': [{'steps': [], 'distance': 590.6, 'duration': 43.9, 'summary': '', 'weight': 43.9}], 'distance': 590.6, 'duration': 43.9, 'weight_name': 'routability', 'weight': 43.9}], 'waypoints': [{'hint': 'CAsigP___38BAAAAAgAAAAUAAAADAAAACv5UQHNkKD-A5AZBf0zJQAEAAAACAAAABQAAAAMAAACmBAAAyEmSBy4sPAK0SZIHViw8AgEATwRjvHkX', 'distance': 4.778892, 'name': '서초대로', 'location': [127.027656, 37.497902]}, {'hint': 'd84FgDapCIB6AAAANgAAAAAAAAAAAAAAXZOqQohpE0IAAAAAAAAAAHoAAAA2AAAAAAAAAAAAAACmBAAApF-SB_IuPAIoX5IHzS48AgAAPxVjvHkX', 'distance': 11.712154, 'name': '테헤란로14길', 'location': [127.033252, 37.49861]}]}


In [20]:
import pandas as pd

In [29]:
# CSV 파일 읽기 및 ID 추가
data = pd.read_csv('final_corner.csv')
data['id'] = data.index + 1  # ID 추가


In [32]:
data.to_csv('marker_id.csv')

In [25]:
import pandas as pd
import itertools
import requests


# 좌표와 ID 준비
locations = list(zip(data['id'], data['경도'], data['위도']))

# OSRM 서버 설정
osrm_server = "http://localhost:5050"
profile = "foot"  # 도보 프로필

# 결과 저장을 위한 리스트
distances = []

# 모든 (시작, 끝) 조합 생성
combinations = itertools.combinations(locations, 2)  # 2개씩 조합

for (id_start, lon_start, lat_start), (id_end, lon_end, lat_end) in combinations:
    # 좌표를 문자열로 변환
    start = f"{lon_start},{lat_start}"
    end = f"{lon_end},{lat_end}"
    url = f"{osrm_server}/route/v1/{profile}/{start};{end}?overview=false"

    # OSRM 요청 및 응답 처리
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("code") == "Ok":
            # 거리 추출
            distance = data["routes"][0]["distance"]
            distances.append({
                "id_start": id_start,
                "id_end": id_end,
                "distance": distance
            })
    else:
        print(f"OSRM 요청 실패: {id_start} -> {id_end}")

# DataFrame 생성
dobo_dist = pd.DataFrame(distances)

# 저장 및 확인
dobo_dist.to_csv("dobo_dist.csv", index=False)
print("도보 거리 계산 완료, 결과 저장: dobo_dist.csv")


도보 거리 계산 완료, 결과 저장: dobo_dist.csv
